In [11]:
%load_ext autoreload
%autoreload 2

In [16]:
from data import dataloader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [19]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [20]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))


none: 19 on-target samples in 2 iteration blocks
environment: 106 on-target samples in 4 iteration blocks
body: 101 on-target samples in 2 iteration blocks
cup: 33 on-target samples in 3 iteration blocks
food: 35 on-target samples in 4 iteration blocks
nails: 69 on-target samples in 3 iteration blocks
smoke: 92 on-target samples in 3 iteration blocks
eyebrow: 235 on-target samples in 5 iteration blocks
nose: 249 on-target samples in 4 iteration blocks
above-ear: 403 on-target samples in 5 iteration blocks
behind-ear: 220 on-target samples in 4 iteration blocks
opposite-cheek: 74 on-target samples in 6 iteration blocks
chin: 166 on-target samples in 1 iteration block
cheek: 686 on-target samples in 6 iteration blocks
forehead: 176 on-target samples in 1 iteration block
top-head: 160 on-target samples in 1 iteration block
back-head: 170 on-target samples in 1 iteration block
opposite-face: 381 on-target samples in 2 iteration blocks
paint-mouth: 81 on-target samples in 3 iteration blocks

In [21]:
pilot_true = []
pilot_false = []
for row in pilot_data[(pilot_data.target == 'above-ear') & (pilot_data.ontarget)][['thermopile1','thermopile2','thermopile3','thermopile4']].values.tolist():
    pilot_true.append({'in':row, 'out':1})
for row in pilot_data[(pilot_data.target == 'chin') & (pilot_data.ontarget == False)][['thermopile1', 'thermopile2', 'thermopile3', 'thermopile4']].values.tolist():
    pilot_false.append({'in':row, 'out':0})

In [22]:
on_data = []
on_targets = []

for piles in pilot_true:
    on_data.append([float(x) for x in piles['in']])
    on_targets.append(piles['out'])

off_data = []
off_targets = []

for piles in pilot_false:
    off_data.append([float(x) for x in piles['in']])
    off_targets.append(piles['out'])

In [67]:
train_size = 50

train_data = []
train_data_p = []
test_data = []
test_data_p = []
train_targets = []
test_targets = []

train_data_p.extend(on_data[:train_size])
train_data_p.extend(off_data[:train_size])

for row in train_data_p:
    row = [np.round(float(x)/98.6,3) for x in row]
    train_data.append(row)

test_data_p.extend(on_data[train_size:])
test_data_p.extend(off_data[train_size:])

for row in test_data_p:
    row = [np.round(float(x)/98.6,3) for x in row]
    test_data.append(row)

train_targets.extend(on_targets[:train_size])
train_targets.extend(off_targets[:train_size])
test_targets.extend(on_targets[train_size:])
test_targets.extend(off_targets[train_size:])

In [69]:
train_data

[[0.83599999999999997,
  0.86799999999999999,
  0.84499999999999997,
  0.85299999999999998],
 [0.83599999999999997,
  0.85199999999999998,
  0.85599999999999998,
  0.83599999999999997],
 [0.82799999999999996,
  0.85199999999999998,
  0.85599999999999998,
  0.82599999999999996],
 [0.82699999999999996,
  0.84799999999999998,
  0.85599999999999998,
  0.82599999999999996],
 [0.82799999999999996,
  0.84799999999999998,
  0.85699999999999998,
  0.82599999999999996],
 [0.82699999999999996,
  0.84799999999999998,
  0.85999999999999999,
  0.82799999999999996],
 [0.82699999999999996,
  0.84799999999999998,
  0.85999999999999999,
  0.82999999999999996],
 [0.82799999999999996,
  0.84799999999999998,
  0.85699999999999998,
  0.82799999999999996],
 [0.82799999999999996,
  0.84799999999999998,
  0.85699999999999998,
  0.82799999999999996],
 [0.82599999999999996,
  0.84799999999999998,
  0.85999999999999999,
  0.83099999999999996],
 [0.82699999999999996, 0.84999999999999998, 0.87, 0.83099999999999996]

In [58]:
clf = MLPClassifier(solver='adam', alpha=.001,
                    hidden_layer_sizes=(25,15,5), random_state=1)
clf.fit(train_data, train_targets)

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 15, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [25]:
num_predict = 300

predictions = clf.predict(test_data[:num_predict])
accuracy_score(test_targets[:num_predict], predictions)

0.0

In [27]:
print(test_data[0])
print(clf.predict(test_data[0]))
print(test_targets[0])

[80.6, 82.5, 83.0, 80.5]
[0]
1


/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
